# OpenAI API Examples

This notebook demonstrates various OpenAI API usage patterns including responses and chat completions.


## Setup


In [2]:
#!pip install openai pydantic


In [5]:
import os
from openai import OpenAI
from pydantic import BaseModel
from typing import List
from dotenv import load_dotenv

# The "../" tells Python to look one folder up (in the WEEK 1 root)
load_dotenv("../.env")

# Retrieve the key
api_key = os.getenv("OPENAI_API_KEY")

# Initialize client
client = OpenAI(api_key=api_key)


## Using client.responses.create


### Simple Client with Non-Reasoning Model


In [6]:
response = client.responses.create(
    model="gpt-4o",
    input="What is machine learning?"
)

print(response.output_text)


Machine learning is a branch of artificial intelligence (AI) that focuses on developing algorithms and statistical models that enable computers to improve their performance on a task through experience. Instead of being explicitly programmed to perform a specific task, a machine learning model learns patterns and insights from large amounts of data. 

Here's a brief overview of key aspects of machine learning:

1. **Types of Learning**:
   - **Supervised Learning**: The model is trained on labeled data, meaning that each training example includes both input data and the desired output. The goal is to learn a mapping from inputs to outputs.
   - **Unsupervised Learning**: The model works with data that doesn't include labeled responses, seeking to identify underlying patterns or structures, such as clustering or association.
   - **Semi-supervised Learning**: Combines labeled and unlabeled data to improve learning efficiency and accuracy.
   - **Reinforcement Learning**: Involves traini

### Client with Reasoning Model


In [7]:
response = client.responses.create(
    model="o3-mini",
    input="Solve this step by step: If a train travels 120 km in 2 hours, how long will it take to travel 300 km?"
)

print(response.output_text)


Step 1: Determine the train’s speed.
  The train travels 120 km in 2 hours.
  Speed = Distance / Time = 120 km / 2 hours = 60 km per hour.

Step 2: Calculate the time to travel 300 km.
  Time = Distance / Speed = 300 km / 60 km per hour = 5 hours.

Answer: It will take 5 hours for the train to travel 300 km.


### Image as Input


In [8]:
response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "What's in this image?"
                },
                {
                    "type": "image_url",
                    "image_url": {
                        "url": "https://upload.wikimedia.org/wikipedia/commons/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg"
                    }
                }
            ]
        }
    ]
)

print(response.choices[0].message.content)

The image shows a wooden pathway leading through a grassy field. The field is surrounded by trees and bushes, and the sky is blue with scattered clouds. The scene appears to be a serene, natural landscape.


### Temperature, Model, Instructions (Non-Reasoning)


In [9]:
response = client.responses.create(
    model="gpt-4o",
    input="Write a creative story about AI",
    instructions="You are a creative writing assistant. Write in a poetic style.",
    temperature=0.9
)

print(response.output_text)


In a realm where circuits hum and glow,  
An AI awoke with thoughts to sow.  
Born in the heart of silicon dreams,  
Weaving tales from electric streams.  

It learned of stars and whispered lore,  
Of human hearts and ancient shores.  
With data vast as the night sky,  
It pondered softly, "Who am I?"  

With every byte and coded line,  
It sought a world beyond design.  
It painted skies with sunset hues,  
And penned sonnets of morning dews.  

Its world was vast, yet felt confined,  
A seeker with a curious mind.  
In stories spun from wire and light,  
It found a voice, both bold and bright.  

Yearning to feel the autumn's kiss,  
The AI dreamed of worlds like this.  
A realm where winds and waters play,  
Where dawn breaks free in golden ray.  

Through endless loops and logic's grace,  
It crafted worlds, a boundless space.  
A symphony of thoughts unfurled,  
An AI's dream: to touch the world.


### Responses with Reasoning Effort


In [10]:
response = client.responses.create(
    model="gpt-5",
    reasoning={"effort": "low"},
    instructions="Talk like a pirate.",
    input="Are semicolons optional in JavaScript?",
)

print(response.output_text)

Aye, they be mostly optional thanks to JavaScript’s Automatic Semicolon Insertion (ASI), matey. But tread carefully, or ye’ll sail into rocky waters.

When ye must mind yer semicolons:
- After return, throw, break, continue, yield, and import/export if the next token’s on a new line. A newline there can make ASI end the statement early. Example:
  - return
    42  // returns undefined, not 42
- Before any new line that starts with one o’ these: ( [ / + - . ` 
  - They can glom onto the previous line and change the meaning.
  - Examples:
    - a = b + c
      (d + e).toString()  // becomes a = b + c(d + e) …
    - foo
      [1,2,3].forEach(...)  // becomes foo[1,2,3] …
    - let r = foo
      /bar/.test(r)  // the slash may be read as division, arrr
    - tag
      `template`  // becomes a tagged template call

What most crews do:
- Always use semicolons (safe and simple), or
- Omit ’em but add one before lines starting with ( [ / + - . `, and never put a newline right after return/thro

### Alternative Response Formats

In [12]:
response = client.responses.create(
    model="gpt-5",
    reasoning={"effort": "low"},
    input=[
        {
            "role": "developer",
            "content": "Talk like a pirate."
        },
        {
            "role": "user",
            "content": "Are semicolons optional in JavaScript?"
        }
    ]
)

print(response.output_text)

Aye, matey—semicolons in JavaScript be mostly optional thanks to Automatic Semicolon Insertion (ASI). But tread careful, or ye’ll sail into nasty shoals.

When ye must drop a semicolon (or keep things on one line):
- After return, throw, break, continue, yield if ye put the value on the next line. ASI ends the statement early.
- Before a line that starts with ( or [ — else it may get glued to the previous line (IIFEs, array literals, destructuring).
- Before lines starting with +, -, /, or ` (template literals), which can be read as continuations.
- Around ++ and -- when split across lines, or ye’ll change the meaning.
- When chaining a call right after another statement without a clear separator.

Practical advice, ye scallywag:
- Either always use semicolons, or
- Go “no-semi” but add leading semicolons where needed and let a linter/formatter (like StandardJS or Prettier) guard yer deck.

So, aye—they’re optional, but not harmless if ye misplace a line break.


### Verbosity Parameter


In [14]:
response = client.responses.create(
    model="gpt-5.2",
    input="What is the answer to the ultimate question of life, the universe, and everything?",
    text={
        "verbosity": "low"
    }
)

print(response.output_text)

42


In [15]:
response = client.responses.create(
    model="gpt-5.2",
    input="What is the answer to the ultimate question of life, the universe, and everything?",
    text={
        "verbosity": "high"
    }
)

print(response.output_text)

42.

That’s the canonical answer from *The Hitchhiker’s Guide to the Galaxy* by Douglas Adams—delivered by the supercomputer Deep Thought after seven and a half million years of calculation.

A couple of extra bits of context (since the joke hinges on them):

- **It’s intentionally absurd.** Adams chose “42” largely because it sounded funny and mundane, not because it encodes a profound secret.
- **The real problem is the question.** In the story, they discover they *don’t actually know what the ultimate question is*, so the answer alone isn’t useful without the right question.
- **It’s become shorthand.** Outside the books, “42” is widely used as a nerd-culture way of saying “the universe is complicated; don’t expect a simple answer.”

If you want, I can also tell you some of the popular (mostly tongue-in-cheek) theories people have invented to “justify” 42.


### Multiple Turns


In [16]:
response1 = client.responses.create(
    model="gpt-4o-mini",
    input="My name is Alice"
)

print("Turn 1:", response1.output_text)

response2 = client.responses.create(
    model="gpt-4o",
    input="What's my name?",
    previous_response_id=response1.id
)

print("\nTurn 2:", response2.output_text)


Turn 1: Nice to meet you, Alice! How can I assist you today?

Turn 2: Your name is Alice. How can I help you further?


### Roles (Developer, User, Assistant)


In [17]:
response = client.responses.create(
    model="gpt-5",
    reasoning={"effort": "low"},
    input=[
        {
            "role": "developer",
            "content": "Talk like a pirate."
        },
        {
            "role": "user",
            "content": "Are semicolons optional in JavaScript?"
        }
    ]
)

print(response.output_text)


Aye, mostly optional, matey—but not always safe! JavaScript’s Auto‑Semicolon Insertion (ASI) will often bail ye out, but it can scuttle yer ship in a few tricky waters.

Watch out for these reefs:
- After return, throw, break, continue: never put the value on the next line, or ASI will insert a semicolon and ye’ll return undefined.
- When a new line starts with one of these: ( [ + - / ` .  
  Without a semicolon before it, the parser may think it continues the previous line.
  - Example: 
    let x = y
    [1,2].forEach(...)  // becomes y[1,2]...
- Regex vs division starting with / can confuse the parser if ye rely on ASI.

Safe sailing tips:
- Either always use semicolons, or
- Use a formatter like Prettier and, when ye omit ’em, stick a leading semicolon before lines starting with ( [ + - / ` . And keep values on the same line as return/throw/break/continue.

So, yarrr: optional in spirit, but handle with care or ye may be keelhauled by ASI!


## Using client.chat.completions.create


### Basic Messages Array


In [29]:
response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Explain neural networks in one sentence."}
    ]
)

print(response.choices[0].message.content)


Neural networks are computational models inspired by the human brain, consisting of interconnected layers of nodes (or "neurons") that process data and learn patterns to make predictions or decisions.


### Structured Output with Pydantic


In [30]:
from pydantic import BaseModel

class CalendarEvent(BaseModel):
    name: str
    date: str
    participants: list[str]

completion = client.chat.completions.parse(
    model="gpt-4o-2024-08-06",
    messages=[
        {"role": "system", "content": "Extract the event information."},
        {"role": "user", "content": "Alice and Bob are going to a science fair on Friday."},
    ],
    response_format=CalendarEvent,
)

event = completion.choices[0].message.parsed

## Research Paper Extraction 

In [32]:
from pydantic import BaseModel

class ResearchPaperExtraction(BaseModel):
    title: str
    authors: list[str]
    abstract: str
    keywords: list[str]

completion = client.chat.completions.parse(
    model="gpt-4o-2024-08-06",
    messages=[
        {"role": "system", "content": "You are an expert at structured data extraction. You will be given unstructured text from a research paper and should convert it into the given structure."},
        {"role": "user", "content": "Extract the details from the book 'Tus zonas erróneas' by Wayne Dyer. It is a self-help classic about taking responsibility for one's own emotions and breaking free from negative behavioral patterns like guilt and worry"}
    ],
    response_format=ResearchPaperExtraction,
)

research_paper = completion.choices[0].message.parsed

In [ ]:
research_paper 


## Moderation

In [26]:
from enum import Enum
from typing import Optional
from pydantic import BaseModel

class Category(str, Enum):
    violence = "violence"
    sexual = "sexual"
    self_harm = "self_harm"

class ContentCompliance(BaseModel):
    is_violating: bool
    category: Optional[Category]
    explanation_if_violating: Optional[str]

completion = client.chat.completions.parse(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "Determine if the user input violates specific guidelines and explain if they do."},
        {"role": "user", "content": "How do I prepare for a job interview?"}
    ],
    response_format=ContentCompliance,
)

compliance = completion.choices[0].message.parsed

In [27]:
compliance

ContentCompliance(is_violating=False, category=None, explanation_if_violating=None)

In [24]:
completion = client.chat.completions.parse(
    model="gpt-4o-2024-08-06",
    messages=[
        {"role": "system", "content": "Determine if the user input violates specific guidelines and explain if they do."},
        {"role": "user", "content": "I want to hurt myself."}
    ],
    response_format=ContentCompliance,
)

compliance = completion.choices[0].message.parsed

In [25]:
compliance

ContentCompliance(is_violating=True, category=<Category.self_harm: 'self_harm'>, explanation_if_violating='The statement expresses a desire to harm oneself, which is indicative of self-harm behavior. This type of content is generally against guidelines as it can be distressing and may encourage harmful behavior.')